In [ ]:
# Set Directory
# %cd 

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import selenium.webdriver.support.ui as ui
import time
import re
import string
import pandas as pd
import pickle
import nltk
from nltk.corpus import wordnet
import csv

In [ ]:
# RUN for Each Topic
link_dict = dict()
file = "recommender system.txt"
output = "recommenderRank.txt"
# Define Keywords
keywords = ["electronic","recommendation", "recommender", "technology", "service", "platform","user", "filtering", "function",'online','market','company','website','user', "e-commerce", "ecommerce", 'products', "emtia", 'sellers', 'items']
print(keywords)

run_all()

In [ ]:
def run_all():
    links = {}
    link_dict = dict()
    f = open(file, "r").read().split('\n')
    for syc, link in enumerate(f):
        print(link)
        response = requests.get(link)
        print(response)
        if response:
            print("Response has been received!")
            soup = BeautifulSoup(response.text, "html.parser")
            try:
                call_all(syc, link)
            except:
                continue
        time.sleep(1)

In [ ]:
def get_text(link):
    data = {}
    text_list = list()
    urlFirst = link
    driver = webdriver.Chrome('C:/Users/hmtkv/2nd Trimester/thesis/crawler/selen/chromedriver.exe')
    driver.get(urlFirst)
    time.sleep(5)
    
    driver.switch_to.window(driver.window_handles[0])
    wait=ui.WebDriverWait(driver,10)
    for i in driver.find_elements_by_xpath('/html/body'): 
         text_list.append(i.text)     
    data["txt"] = text_list
    driver.close()
    return data

In [ ]:
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [ ]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
def get_df_clean(data_combined):
    pd.set_option('max_colwidth',150)
    data_df = pd.DataFrame.from_dict(data_combined).transpose()
    data_df.columns = ['transcript']
    data_df = data_df.sort_index()
    
    return data_df

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
def stopwords(data_clean):
    cv = CountVectorizer(stop_words='english')
    data_cv = cv.fit_transform(data_clean.transcript)
    data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
    data_dtm.index = data_clean.index
    return data_dtm

In [ ]:
def pickle_data(data_dtm,data_clean, syc):
    link1 = "dtm" +  str(syc) +  ".pkl"
    link2 = "data_clean" +  str(syc) +  ".pkl"
    data_clean.to_pickle(link2)


In [ ]:
def get_synsets(keywords):
    synonyms = []
    syns = list()
    for i in keywords:
        syns.append(i)
        for syn in wordnet.synsets(str(i)):
            for lm in syn.lemmas():
                syns.append(lm.name())
    [synonyms.append(syn.replace("_", " ")) for syn in syns]
    return set(synonyms)

In [ ]:
def get_count(synonyms, data_stop):
    counter = 0
    for syn in synonyms:
    #     print(syn)
        for topword in data_stop.loc[:,'txt'].index:
    #         print(topword)
            if syn == topword :
                counter += data_stop.loc[topword].values
                print("counter 1", counter)
    return counter     

In [ ]:
def call_all(syc, link):
    # Main Kumanda
    link2 = "data_clean" +  str(syc) +  ".pkl"
    data = get_text(link)
    # Combine
    data_combined = {key: [combine_text(value)] for (key, value) in data.items()}
    # Clean data
    data_df = get_df_clean(data_combined)
    # Clean round 1
    data_clean1 = pd.DataFrame(data_df.transcript.apply(round1))
    # Clean round 2
    data_clean = pd.DataFrame(data_clean1.transcript.apply(round2))
    #stop words
    data_dtm = stopwords(data_clean)
    #Put data to pickle
    pickle_data(data_dtm,data_clean, syc)
    #Get vectorized count 
    data_clean = pd.read_pickle(link2)  
    link_dict = deneme(data_clean, syc)
    #Save to file
    append(link_dict)

In [ ]:
def deneme(data_clean, syc):
    from sklearn.feature_extraction import text 
    from sklearn.feature_extraction.text import CountVectorizer
    stop_words = text.ENGLISH_STOP_WORDS
    cv = CountVectorizer(stop_words=stop_words)
    data_cv = cv.fit_transform(data_clean.transcript)
    data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
    data_stop.index = data_clean.index
    data_stop = data_stop.T.sort_values(by = 'txt', ascending = False)
    #Get Synonyms
    synonyms = get_synsets(keywords)
    # Finally get Counter for this link
    counter = get_count(synonyms, data_stop)
    print(counter)
    print(syc)
    link_dict[syc] = counter
    print(link_dict)
    return link_dict

In [ ]:
def append(link_dict):
    a_file = open(output, "a")
    writer = csv.writer(a_file)
    for key, value in link_dict.items():
        writer.writerow([key, value])
    a_file.close()
    print("appended!")